# ACL Playlist Creation Code

#### The purpose of this workbook is to create a playlist of all of the artists who have played at Austin City Limits.

#### The Artists are stored in a CSV file and imported into the notebook. Using Spotipy, the program searches Spotify for each artist listed in the code. Once found, the code adds the Artist's top song into a created playlist. 

##### Initialize modules

In [1]:
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import spotipy
import pandas as pd

##### Initialize Spotify Developer Tools for playlist modification


In [2]:
client_id = "hidden" # I've hidden my ID in order to publish this code
client_secret = "hidden"# I've hidden my secret in order to publish this code
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

token = util.prompt_for_user_token(
        username='1229757239',
        scope='playlist-modify-public',
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri='http://localhost:8888/callback')

sp = spotipy.Spotify(auth=token,client_credentials_manager=client_credentials_manager)

##### Create a dataframe from the Excel file and drop all duplicates for quicker processing

In [4]:
df = pd.read_excel("ACL_Artists_For_Playlist_Creation.xlsx")
df = df.drop_duplicates(subset=['Artist'])

##### Use Spotipy to search for artists and create a playlist based on their top songs

In [6]:
artist_uri_list = []
track_uri_list = []

for artist in df['Artist']:
    # Search for artist and store their JSON info in a variable
    artist_info = sp.search(artist,limit=1, type='artist')
    # Set any unfound artists to 0 for manual addition
    if artist_info['artists']['total'] == 0:
        artist_uri = 0
        top_track_name = ''
        top_track_uri = ''
    # For all found artists, store their JSON top track info in a variable
    else:
        artist_uri = artist_info['artists']['items'][0]['uri']
        top_track = sp.artist_top_tracks(artist_uri)
        # Set any artists without top tracks to 0 for manual addition
        if top_track['tracks'] == []:
            top_track_name = ''
            top_track_uri = ''
        # For all found tracks, add the top track to a list
        else:
            top_track_name = top_track['tracks'][0]['name']
            top_track_uri = top_track['tracks'][0]['id']
            artist_uri_list.append([artist, artist_uri,top_track_name, top_track_uri])
            track_uri_list.append(top_track_uri)
            # Once the list reaches 50 tracks, use Spotipy to add the tracks to the playlist and clear the list
            if len(track_uri_list) == 50:
                sp.user_playlist_add_tracks(user='1229757239',playlist_id='0TCMt3rZMXJfOfvNBDf34E',tracks=track_uri_list)
                track_uri_list = []